# Mnist Fashion Test

In [1]:
import os
import urllib
import urllib.request
import cv2

import numpy as np
import matplotlib.pyplot as plt

from zipfile import ZipFile
import sys
sys.path.insert(1, r'../src')


import model

from utils import one_hot_encode_index

from optimizers import Adam
from activations import Softmax, ReLU
from layers import Dropout, LinearLayer
from loss import CategoricalCrossEntropyLoss

## Data Preparation 

In [2]:
URL = 'https://nnfs.io/datasets/fashion_mnist_images.zip'
FILE = 'fashion_mnist_images.zip'
FOLDER = 'fashion_mnist_images'

if not os.path.isfile(FILE):
    print(f'Downloading {URL} and saving as {FILE}...')
    urllib.request.urlretrieve(URL, FILE)

print('Unzipping images...')
with ZipFile(FILE) as zip_images:
    zip_images.extractall(FOLDER)

print('Done!')


Unzipping images...
Done!


## Loading Data 

In [2]:
# Loads a MNIST dataset
def load_mnist_dataset(dataset, path):
    # Scan all the directories and create a list of labels
    labels = os.listdir(os.path.join(path, dataset))
    # Create lists for samples and labels
    X = []
    y = []
    # For each label folder
    for label in labels:
        # And for each image in given folder
        for file in os.listdir(os.path.join(path, dataset, label)):
            # Read the image
            image = cv2.imread(os.path.join(path, dataset, label, file), cv2.IMREAD_UNCHANGED)
            # And append it and a label to the lists
            X.append(image)
            y.append(label)
    # Convert the data to proper numpy arrays and return
    return np.array(X), np.array(y).astype('uint8')


def create_data_mnist(path):
    # Load both sets separately
    X, y = load_mnist_dataset('train', path)
    X_test, y_test = load_mnist_dataset('test', path)
    y = one_hot_encode_index(y, 10)
    y_test = one_hot_encode_index(y_test, 10)
    
    # And return all the data
    return X, y, X_test, y_test


In [3]:
X, y, X_test, y_test = create_data_mnist('fashion_mnist_images')


## Preprocess data

### Scaling between -1 & 1

In [4]:
def scale_img(v):
    return (v - 127.5) /127.5

X = scale_img(X)
X_test = scale_img(X_test)

### Reshaping data 

In [5]:
def vectorize(v):
    return v.reshape(v.shape[0], -1)

In [6]:
X = vectorize(X)
X_test = vectorize(X_test)

### Shuffle Training data 

In [7]:
keys = np.array(range(X.shape[0]))
np.random.shuffle(keys)
X = X[keys]
y = y[keys]

In [15]:
## TESTING 
import importlib
importlib.reload(model)
##

cce_loss = CategoricalCrossEntropyLoss()
optimizer = Adam(decay=5e-5)

my_model = model.Model(optimizer, cce_loss)

my_model.set_sequence([
    LinearLayer(X.shape[1], 128),
    ReLU(),
    Dropout(0.5),
    LinearLayer(128, 128),
    ReLU(),
    LinearLayer(128, 10),
    Softmax()
])
my_model.train(X, y, epochs=10, batch_size=128, validation=(X_test, y_test))

=== Epoch: 1 ===
Step: 0/469, accuracy0.062, loss3.100, learning rate 0.0010000 
Step: 100/469, accuracy0.711, loss0.723, learning rate 0.0009950 
Step: 200/469, accuracy0.742, loss0.600, learning rate 0.0009901 
Step: 300/469, accuracy0.766, loss0.609, learning rate 0.0009852 
Step: 400/469, accuracy0.789, loss0.500, learning rate 0.0009804 
Step: 468/469, accuracy0.833, loss0.454, learning rate 0.0009771 
Epoch: 0/10, accuracy0.750, loss0.694, learning rate 0.001
--Validation--
Validation : Accuracy: 0.828, Loss: 0.473
=== Epoch: 2 ===
Step: 0/469, accuracy0.812, loss0.578, learning rate 0.0009771 
Step: 100/469, accuracy0.766, loss0.502, learning rate 0.0009723 
Step: 200/469, accuracy0.867, loss0.424, learning rate 0.0009676 
Step: 300/469, accuracy0.820, loss0.464, learning rate 0.0009630 
Step: 400/469, accuracy0.812, loss0.417, learning rate 0.0009584 
Step: 468/469, accuracy0.885, loss0.332, learning rate 0.0009552 
Epoch: 1/10, accuracy0.815, loss0.512, learning rate 0.001
--V

In [16]:
my_model.evaluate(X_test, y_test, 128)

Validation : Accuracy: 0.867, Loss: 0.359
